In [1]:
.open tmp.db
.load ../../dist/vec0
.load ./rembed0

insert into rembed_clients(name, options)
  values ('snowflake-arctic-embed-m-v1.5', 'llamafile');

--select vec_version(), rembed_debug();

[no code]

## FTS Search

In [10]:
.param set query planned parenthood

select
  rowid,
  highlight(fts_headlines, 0, '<b>', '</b>') as headline_highlighted,
  rank
from fts_headlines
where headline match :query
order by rank
limit 10;

rowid,headline_highlighted,rank
4666,Kamala Harris visits <b>Planned</b> <b>Parenthood</b> clinic,-18.9139950477264
6521,Former Marine sentenced to 9 years in prison for firebombing <b>Planned</b> <b>Parenthood</b> clinic,-14.807022703838651


##  Vector Search

In [11]:
.param set query planned parenthood

select
  article_id,
  articles.headline,
  distance
from vec_headlines
left join articles on articles.rowid = vec_headlines.article_id
where headline_embedding match vec_normalize(vec_slice(rembed('snowflake-arctic-embed-m-v1.5', :query), 0, 256))
  and k = 10;

SQL logic error: Error sending HTTP request: http://localhost:8080/embedding: Connection Failed: Connect error: Connection refused (os error 61)

## FTS + Vector search: RFF

In [13]:
.param set query planned parenthood

insert or replace into temp.sqlite_parameters
  values ('query_embedding', rembed('snowflake-arctic-embed-m-v1.5', :query));

.param set weight_fts 1.0
.param set weight_vec 1.0
.param set rrf_k 60
.param set k 10


with vec_matches as (
  select
    article_id,
    row_number() over (order by distance) as rank_number,
    distance
  from vec_headlines
  where
    headline_embedding match vec_slice(:query_embedding, 0, 256)
    and k = :k
  order by distance
),
fts_matches as (
  select
    rowid,
    row_number() over (order by rank) as rank_number,
    rank as score
  from fts_headlines
  where headline match :query
  limit :k
),
final as (
  select
    articles.id,
    articles.headline,
    vec_matches.distance as vector_distance,
    fts_matches.score as fts_score,
    coalesce(1.0 / (:rrf_k + fts_matches.rowid), 0.0) * :weight_fts +
      coalesce(1.0 / (:rrf_k + vec_matches.article_id), 0.0) * :weight_vec
      as combined_score

  from fts_matches
  full outer join vec_matches on vec_matches.article_id = fts_matches.rowid
  join articles on articles.rowid = coalesce(fts_matches.rowid, vec_matches.article_id)
  order by combined_score desc
)
select * from final;



SQL logic error: SQL logic error (1) : Error sending HTTP request: http://localhost:8080/embedding: Connection Failed: Connect error: Connection refused (os error 61)

In [8]:
select json('[1,2,3,4]'), vec_f32(X'AABBCCDD')

"json('[1,2,3,4]')",vec_f32(X'AABBCCDD')
"[1,2,3,4]",Blob<4>


In [9]:
select * from bytecode('select count(*) from pragma_table_list;')

addr,opcode,p1,p2,p3,p4,p5,comment,subprog,nexec,ncycle
0,Init,0,12,0,,0,,,0,0
1,Null,0,1,1,,0,,,0,0
2,VOpen,0,0,0,vtab:7FAC27505B30,0,,,0,0
3,Integer,0,2,0,,0,,,0,0
4,Integer,0,3,0,,0,,,0,0
5,VFilter,0,8,2,,0,,,0,0
6,AggStep,0,0,1,count(0),0,,,0,0
7,VNext,0,6,0,,0,,,0,0
8,AggFinal,1,0,0,count(0),0,,,0,0
9,Copy,1,4,0,,0,,,0,0
